In [1]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import cv2
import mediapipe as mp
import torch
import pandas as pd
from scipy.fft import fft, ifft


In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
torch.cuda.empty_cache()

Using cuda device


In [3]:
def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [4]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [5]:

def getFiturLandmark(img):
    err = None
    detection_result = []
    try:
        
        base_options = python.BaseOptions(model_asset_path='pose_landmarker_heavy.task')
        options = vision.PoseLandmarkerOptions(
        base_options=base_options,
        output_segmentation_masks=True)
        detector = vision.PoseLandmarker.create_from_options(options)
        
        detection_result = detector.detect(img)

        err = None
    
    except:
        err = 1
        detection_result = []
    
    return detection_result, err



    
    

In [6]:
# ainput = mp.Image.create_from_file("../../dataset/extractFrame/above/00431/121.jpg")
# aaa, aer = getFiturLandmark(ainput)
# aaalandmarklist = aaa.pose_landmarks
# # _dfpose = pd.DataFrame(aaalandmarklist[0])
# print(len(aaalandmarklist))

In [7]:
# concate 
def flatten(data):
    a = np.ravel(data)
    return a

In [8]:
# feed to the dataset huhuyy

import os
import pandas as pd
from scipy.fft import fft, ifft


def load_feature_from_folder(folder):
  img = []
  data = []
  for folderName in os.listdir(folder):
    print(folderName)
    folder1 = os.path.join(folder, folderName)
    for foll in os.listdir(folder1):
      print(foll)
      folder2 = os.path.join(folder1, foll)
      cnt = 0
      for fileInside in os.listdir(folder2):
        pathLengkap = os.path.join(folder2, fileInside)
        namaClass = folderName
        mp_image = mp.Image.create_from_file(pathLengkap)
        result, err = getFiturLandmark(mp_image)
        
        
        if(err == None):
          try:
            pose_landmarks_list = result.pose_landmarks
            dfpose = pd.DataFrame(pose_landmarks_list[0])
            print(cnt)
            print(pose_landmarks_list[0])
            hasil1 = flatten(dfpose)
            hasilfft = np.fft.fft(hasil1)
            masuk_data = {
              'class': namaClass,
              'namaFile': foll,
              'urutan': cnt,
              'feature_lengkap': dfpose,
              'triangle_feature': hasil1,
              'fourier_feature': hasilfft
            }
            cnt = cnt + 1
            data.append(masuk_data)
            # print(data)
          except:
            print(pathLengkap)
        else:
          continue
  dataFrame = pd.DataFrame(data)
  dataFrame.to_pickle("data_keypoint_fitur_lengkap_pickle")
         
path_dataset = "../../dataset/extractFrame"
# load_feature_from_folder(path_dataset)

In [9]:
# buat listfile csv

def createListFile(_folder):
    _data = []
    for _folderName in os.listdir(_folder):
        print(_folderName)
        _folder1 = os.path.join(_folder, _folderName)
        for _foll in os.listdir(_folder1):
            print(_foll)
            _folder2 = os.path.join(_folder1, _foll)
            cnt = 0
            for _fileInside in os.listdir(_folder2):
                _pathLengkap = os.path.join(_folder2, _fileInside)
                _namaClass = os.path.join(_folderName,_foll, _fileInside)
                print(_namaClass)
                _data.append(_namaClass)
    
    _dtframe = pd.DataFrame(_data)
    _dtframe.to_pickle("list_file")

path_dataset = "../../dataset/extractFrame"
# createListFile(path_dataset)

In [10]:
# create dataset class
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from sklearn.cluster import KMeans


class dataClass(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.file = pd.read_pickle("list_file")
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.file)
    
    def __getitem__(self, index):
        _path_depan = self.root_dir
        _path_belakang = self.file.iloc[index,0]
        _path_belakang = _path_belakang.replace(os.sep, '/')
        _path_lengkap = os.path.join(_path_depan, _path_belakang)
        _image = cv2.imread(_path_lengkap)
        _mpImage = mp.Image.create_from_file(_path_lengkap)
        _result, _err = getFiturLandmark(_mpImage)
        _pose_landmarks_list = _result.pose_landmarks
        if(len(_pose_landmarks_list)>0):
            
            _dfpose = pd.DataFrame(_pose_landmarks_list[0])
            _dfposeFlatten = flatten(_dfpose)
            _datareturn = {'path': _path_lengkap,
                       'image': _image,
                       'fitur_keypoint_lengkap': _result,
                       'fitur_lengkap_flatten': _dfposeFlatten,
                       }
        else:
            return self.__getitem__(index=index+1)
        return _datareturn

dataaset_torch = dataClass(csv_file="list_file", root_dir="../../dataset/extractFrame/")   


In [31]:
data_lengkap = pd.read_pickle("../../dataset/data_keypoint_fitur_lengkap_pickle")
# data_lengkap.iloc[0]["triangle_feature"]
a = data_lengkap.iloc[642][3]
a


,x,y,z,visibility,presence
0,0.503690,0.329571,-1.182622,0.999827,0.999845
1,0.529338,0.289014,-1.133515,0.999657,0.999782
2,0.542073,0.290359,-1.134209,0.999594,0.999784
3,0.558434,0.291143,-1.134147,0.999672,0.999673
4,0.484639,0.286153,-1.125305,0.999773,0.999729
5,0.472119,0.286374,-1.126263,0.999795,0.999772
6,0.461666,0.287675,-1.126953,0.999836,0.999693
7,0.577833,0.311764,-0.782664,0.999735,0.999629
8,0.448712,0.306605,-0.744369,0.999589,0.999584
9,0.533603,0.380052,-1.037248,0.999635,0.999551
